In [1]:
import pandas as pd
from collections import Counter
import json
import tqdm
import logging
from nltk import untag
from pymystem3 import Mystem
import nltk
import string
import json, urllib
import pandas as pd
import numpy as np
import glob

In [ ]:
path = '~/Downloads/migrants_data/' # путь до папки с текстами по годам (которые 2015.csv и т.д.)
all_files = glob.glob(path + "*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['year'] = pd.to_numeric(filename.replace('.csv', '')[-4:])
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
data = data[['description', 'year']]
data.columns = ['text', 'year']

### Лемматизация
#### Приводим слова к начальной форме

In [ ]:
m = Mystem()

In [ ]:
lem_text = []
for i in tqdm.tqdm(data['text']):
    try:
        lemmas = m.lemmatize(i)
        lem = ''.join(lemmas)
        lem_text.append(lem)
    except:
        lem_text.append('')

In [ ]:
data['lem_text'] = lem_text

In [ ]:
# на всякий случай сохраняем данные с лемматизированными текстами в csv
data.to_csv('~/Downloads/migrants_data/lem_texts_migration.csv', index = False)

In [ ]:
pattern = '|'.join(['\n', '\s+'])
data['lem_text'] = data['lem_text'].str.replace(pattern, ' ')

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('russian')

In [ ]:
data['lem_text'] = data['lem_text']\
    .apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.head()

### Chunking
#### Устанавливаем правила, по которым выделяются словосочетания из ранее предобработанных текстов

In [ ]:
grammar = ('''
    AS: {<A=m><S>}
    SS: {<S><S>}
    S1: {<S>}
    ''')

chunkParser = nltk.RegexpParser(grammar)

In [ ]:
lem_text = list(data['lem_text'])

In [ ]:
dfs = []
i = 0
for text in tqdm.tqdm(lem_text):
    i = i + 1
    num_text = "text {}".format(i)
    tagged = nltk.pos_tag(nltk.word_tokenize(text), lang='rus')
    tree = chunkParser.parse(tagged)
    as_tag = []
    ss_tag = []
    s_tag = []
    for subtree in tree.subtrees():
        if subtree.label() == 'AS': as_tag.append(" ".join(untag(subtree)).lower())
        if subtree.label() == 'SS': ss_tag.append(" ".join(untag(subtree)).lower())
        if subtree.label() == 'S1': s_tag.append(" ".join(untag(subtree)).lower())
    as_tag_df = pd.DataFrame({'term': as_tag})
    as_tag_df['tag'] = 'as_tag'
    
    ss_tag_df = pd.DataFrame({'term': ss_tag})
    ss_tag_df['tag'] = 'ss_tag'
    
    s_tag_df = pd.DataFrame({'term': s_tag})
    s_tag_df['tag'] = 's_tag'

    df = as_tag_df.append(ss_tag_df)
    df = df.append(s_tag_df)
    df['num_text'] = num_text
    
    dfs.append(df)

final_df = pd.concat(dfs)

In [ ]:
ss_tag = list(final_df[final_df['tag'] == 'ss_tag'].groupby('term').count().sort_values('tag', ascending=False).head(1000)\
    .reset_index()['term'])

as_tag = list(final_df[final_df['tag'] == 'as_tag'].groupby('term').count().sort_values('tag', ascending=False).head(1000)\
        .reset_index()['term'])

s_tag = list(final_df[final_df['tag'] == 's_tag'].groupby('term').count().sort_values('tag', ascending=False).head(1000)\
    .reset_index()['term'])

import itertools
tags = itertools.chain(ss_tag, as_tag, s_tag)

terms = []
for i in tags:
    terms.append(i)

In [ ]:
fin = []
for i in tqdm.tqdm(terms):
    try:
        temp = final_df.query("term == '{}'".format(str(i)))
        fin.append(temp)
    except:
        Exception

In [ ]:
fin_df_with_top_words = pd.concat(fin)

In [ ]:
data = data.reset_index().drop('index', axis=1)

In [ ]:
data['num_text'] = data.index + 1
data['num_text'] = 'text ' + data['num_text'].astype(str)

In [ ]:
data['id'] = data['num_text'].map(lambda x: x.lstrip('text '))
fin_df_with_top_words['id'] = fin_df_with_top_words['num_text'].map(lambda x: x.lstrip('text '))

In [ ]:
date_term = data[['id', 'year']].merge(fin_df_with_top_words, on='id')

In [ ]:
date_term = date_term.merge(term_count, on=['year','term'])
date_term = date_term[['term', 'year', 'id', 'Time']]
date_term.columns = ['term', 'cat', 'id', 'count']
date_term.head(1)

### Файл со списком словосочетаний для каждого года
#### Записываем в файлы по годам словосочетания из текстов

❗Константа **частота встречаемости словосочетания по всему корпусу текстов**

    Чем больше хотим получить слов в анализ, тем меньше будет это значение

In [ ]:
term_frequency = 20

In [ ]:
date_term = date_term[date_term['count'] > term_frequency]
for y in date_term['cat'].unique():
    date_term[date_term['cat'] == y]\
    [['term', 'cat', 'id', 'count']]\
    .drop_duplicates()\
    .to_csv("~/Downloads/migrants_data/migration_terms_{}.csv".format(int(y)), index=False) 
    # сейчас здесь та же папка, где лежали входные тексты